In [1]:
!pip install -q transformers sentence-transformers faiss-cpu
!pip install -q langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 20.4 MB/s eta 0:00:00


## Preparação e transformação


In [2]:
# Texto que o LLM buscará para fazer RAG
documento = "A madame Herta é a gênia N° 83 da sociedade dos gênios. A socidade dos gênios acolhe todas as pessoas que foram escolhidas por Nous. O criador do Nous foi o primeiro gênio a entrar na sociedade."

In [3]:
# Separar o documento em partes menores (como ele é pequeno eu poderia separar manualmente, mas queria achar uma forma automatica):
frases = [(frase.strip() + '.') for frase in documento.split(sep='.') if frase.strip()]
frases

['A madame Herta é a gênia N° 83 da sociedade dos gênios.',
 'A socidade dos gênios acolhe todas as pessoas que foram escolhidas por Nous.',
 'O criador do Nous foi o primeiro gênio a entrar na sociedade.']

In [4]:
# Aqui é a parte de transformação das frases para vetores utilizando um modelo recomendado pelo Gemini para fazer os embenddings
from sentence_transformers import SentenceTransformer

# Modelo SOTA
nome_modelo = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
modelo = SentenceTransformer(nome_modelo)

# Criação dos Embs
frase_embeddings = modelo.encode(frases)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Utilização do FAIS para criar uma biblioteca de maneira eficiente
import faiss

# A dimensão do vetor
d = frase_embeddings.shape[1]

# Cria o FAISS
index = faiss.IndexFlatL2(d)
index.add(frase_embeddings)

## Busca pelo conhecimento para responder ao usuario de forma coerente

In [6]:
# Aqui é a conversão da string/input do usuario para formato de vetor, já que o RAG funciona com base de semelhanças de vetor
pergunta = str(input("> "))
pergunta_tratada = modelo.encode([pergunta])

k = 2
D, I = index.search(pergunta_tratada, k)

ids_frases = I[0]


> Quem foi que criou o Nous?


In [7]:
contexto = [frases[i] for i in ids_frases]

print("--- Frases com maior correlação---")
for i, frase in enumerate(contexto):
    print(f"Rank {i+1} (ID {ids_frases[i]}): {frase}")

--- Frases com maior correlação---
Rank 1 (ID 2): O criador do Nous foi o primeiro gênio a entrar na sociedade.
Rank 2 (ID 1): A socidade dos gênios acolhe todas as pessoas que foram escolhidas por Nous.


## Geração de resposta com LLM

In [8]:
# Definindo o prompt para evitar que o modelo de informações erradas e sempre busque na biblioteca:
def construir_prompt_rag(pergunta, contexto):
    # Converte a lista de chunks em uma única string formatada
    contexto_formatado = "\n".join(contexto)

    prompt = f"Responda à pergunta APENAS com o texto fornecido. Não use conhecimento externo. Responda no idioma PORTUGUÊS. Pergunta: {pergunta} Contexto: {contexto_formatado}"
    return prompt


prompt_final = construir_prompt_rag(pergunta, contexto)

print("--- Prompt Final Enviado ao LLM ---")
print(prompt_final)

--- Prompt Final Enviado ao LLM ---
Responda à pergunta APENAS com o texto fornecido. Não use conhecimento externo. Responda no idioma PORTUGUÊS. Pergunta: Quem foi que criou o Nous? Contexto: O criador do Nous foi o primeiro gênio a entrar na sociedade.
A socidade dos gênios acolhe todas as pessoas que foram escolhidas por Nous.


In [9]:
# Carrega um modelo pequeno de instrução (ex: T5-base)
# O T5 é um modelo Encoder-Decoder bom para tarefas de resumo e Q&A.
from transformers import pipeline
try:
    model_name = "google/flan-t5-base"
    generator = pipeline("text2text-generation", model=model_name)
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}. Certifique-se de que a biblioteca transformers está instalada.")
    generator = None

if generator:
    resposta_bruta = generator(
        prompt_final,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        clean_up_tokenization_spaces=True
    )

    resposta_final = resposta_bruta[0]['generated_text']

    print("\n--- Resposta ---")
    resposta_limpa = resposta_final.replace("Resposta:", "").strip()

    print(resposta_limpa)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Resposta ---
O criador do Nous foi o primeiro gênio
